In [1]:
# new data structure for STOPWATCHER
# build up tables BY VEHICLE - a history of observations of each vehicle
# then can go and reconstruct unique history for each VEHICLE, based on last observeration before it STOPPED APPROACHING each stop in the _stoplist_

In [2]:
from lib.reportcard_helpers import *
import time

In [3]:
def timeit(method):
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        if 'log_time' in kw:
            name = kw.get('log_name', method.__name__.upper())
            kw['log_time'][name] = int((te - ts) * 1000)
        else:
            print '%r  %2.2f ms' % \
                  (method.__name__, (te - ts) * 1000)
        return result
    return timed


In [4]:
source = 'nj'
route = 87

In [5]:
(conn, db) = db_setup(route)

In [6]:
# only get buses APPROACHING A STOP
arrival_query = ('SELECT * FROM stop_predictions \
                WHERE (rd = %s AND pt = "APPROACHING") \
                ORDER BY timestamp;' % route)

In [7]:
df = pd.read_sql_query(arrival_query, conn)

In [8]:
df = timestamp_fix(df)

In [9]:
# 1 group all arrival predictions by unique vehicle-stop combinations
df.groupby(['v','stop_id'])

# 2 gets the keys for unique vehicle-stop combinations
# a list of tuples
# [(u'5403', u'20640'), (u'5704', u'20933'), .....

keys = df.groupby(['v','stop_id']).groups.keys()


In [10]:
@timeit
def make_history(df):

    # THIS IS WAYYYYYYY FASTER !!!

    # history data structure = assign every row in df to a dict keyed to unique vehicle/stop_id instance
    # {
    #  5403_20640: [row1, row15, ...]
    #  5704_20933: [row2, row37, ...]    
    # }

    from collections import defaultdict
    history = defaultdict(list)

    for index,row in df.iterrows():
        key = row['v'] + '_' + row['stop_id']
        history[key].append(row)

    # sort them and slice them

    keepers = defaultdict(list)
    discards = defaultdict(list)

    # iterate over the dict
    for key_copy, arrivals in history.iteritems():

        # sort each arrival list
        arrivals.sort(key=lambda x: x.timestamp)

        #
        #
        # NEED TO GROUPBY ONES THAT HAVE THE SAME GENERAL ARRIVAL WINDOW
        #
        # PROBLEM WITH BELOW IS THAT IT DOESNT WORK WHEN WE HAVE A FULL HISTORY
        # BECAUSE IT DOESNT DETECT EVENTS? e.g. VEHICLE-STOP COMBINATIONS ARE NOT UNIQUE FOR MORE THAN A FEW HOURS
        #
        #   

        # put last row in keepers
        keepers[key_copy].append(arrivals[-1])

        # put everything else in discards
        discards[key_copy].append(arrivals[:-1])
        
        return keepers,discards


In [ ]:
# this isnt working now.... only returning 1?

In [11]:
(keepers,discards) = make_history(df)


'make_history'  557.00 ms


In [12]:
print keepers

defaultdict(<type 'list'>, {u'5739_21124': [pkey                               39568
cars                                    
consist                                 
fd           87 GATES AVE VIA JOURNAL SQ
m                                      1
name                      StopPrediction
pt                           APPROACHING
rd                                    87
rn                                    87
scheduled                          false
stop_id                            21124
stop_name       SUMMIT AVE + PAVONIA AVE
v                                   5739
timestamp            2018-06-06 14:53:07
Name: 2018-06-06 14:53:07, dtype: object]})


In [13]:
print discards

defaultdict(<type 'list'>, {u'5739_21124': [[]]})
